In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 100
l = 10
num_filter = 40
compression = 0.5
dropout_rate = 0.2



####################################################################################################



import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam 
from keras.layers import Input


# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp
  
  
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg
 




def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output
  
  
  
  




# img_height = 32 
# img_width = 32
# channel = 3


input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# Back_Prop_First_Conv2D

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate=0)
First_Transition = add_transition(First_Block, num_filter, dropout_rate=0)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)


# Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
# Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)



model = Model(inputs=[input], outputs=[output])
model.summary()


sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# top_5_accuracy = keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=5)



# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])


filepath = f'/gdrive/My Drive/CIFAR_10/__{epochs}_.h5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='auto', 
                                period=5)

csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)



datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     brightness_range = [0.2 , 0.6],
#     rotation_range=15,
    width_shift_range=0.125,
    height_shift_range=0.125,
     fill_mode='constant',
#     zoom_range=0.2,
    horizontal_flip=True)


datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/100,
                    epochs=epochs , 
                   verbose = 1 , 
#                    workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True)

# model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_test, y_test))






# Save the trained weights in to .h5 format
model.save_weights("version_1_Test_acuracy_0.8616_DNST_model.h5")
print("Saved model to disk")





__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 32, 32, 40)   1080        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_45 (BatchNo (None, 32, 32, 40)   160         conv2d_45[0][0]                  
__________________________________________________________________________________________________
activation_45 (Activation)      (None, 32, 32, 40)   0           batch_normalization_45[0][0]     
__________________________________________________________________________________________________
conv2d_46 

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__100_.h5")


model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                    callbacks=[csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/100,
                    epochs=30 , 
                   verbose = 1 , 
#                    workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))





Epoch 1/30
782/782 [==============================] - 241s 308ms/step - loss: 0.3818 - acc: 0.8668 - val_loss: 0.7583 - val_acc: 0.8081
Epoch 2/30
782/782 [==============================] - 237s 303ms/step - loss: 0.3715 - acc: 0.8700 - val_loss: 0.6519 - val_acc: 0.8222
Epoch 3/30
782/782 [==============================] - 238s 305ms/step - loss: 0.3637 - acc: 0.8735 - val_loss: 0.6595 - val_acc: 0.8304
Epoch 4/30
782/782 [==============================] - 238s 304ms/step - loss: 0.3547 - acc: 0.8778 - val_loss: 0.6298 - val_acc: 0.8171
Epoch 5/30
782/782 [==============================] - 238s 304ms/step - loss: 0.3397 - acc: 0.8827 - val_loss: 0.5769 - val_acc: 0.8460

Epoch 00005: acc did not improve from 0.89532
Epoch 6/30
782/782 [==============================] - 238s 305ms/step - loss: 0.3377 - acc: 0.8840 - val_loss: 0.6445 - val_acc: 0.8235
Epoch 7/30
782/782 [==============================] - 239s 306ms/step - loss: 0.3313 - acc: 0.8840 - val_loss: 0.8150 - val_acc: 0.7903
E

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 18s 2ms/step
Test loss: 0.5870169169366359
Test accuracy: 0.8558


In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__100_.h5")

sgd = keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)


reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1, 
                                  patience=5, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.001)




model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                    callbacks=[reduce_lr , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/100,
                    epochs=20 , 
                   verbose = 1 , 
#                    workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))





Epoch 1/20
782/782 [==============================] - 240s 307ms/step - loss: 0.2232 - acc: 0.9221 - val_loss: 0.7536 - val_acc: 0.8318
Epoch 2/20
782/782 [==============================] - 238s 304ms/step - loss: 0.2292 - acc: 0.9192 - val_loss: 0.6578 - val_acc: 0.8404
Epoch 3/20
782/782 [==============================] - 237s 304ms/step - loss: 0.2169 - acc: 0.9241 - val_loss: 0.6643 - val_acc: 0.8382
Epoch 4/20
782/782 [==============================] - 237s 303ms/step - loss: 0.2145 - acc: 0.9239 - val_loss: 0.7835 - val_acc: 0.8382
Epoch 5/20
782/782 [==============================] - 237s 304ms/step - loss: 0.2128 - acc: 0.9247 - val_loss: 0.4297 - val_acc: 0.8819

Epoch 00005: acc improved from 0.92080 to 0.92458, saving model to /gdrive/My Drive/CIFAR_10/__100_.h5
Epoch 6/20
782/782 [==============================] - 238s 304ms/step - loss: 0.2126 - acc: 0.9248 - val_loss: 0.4650 - val_acc: 0.8811
Epoch 7/20
782/782 [==============================] - 238s 304ms/step - loss: 0.

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 18s 2ms/step
Test loss: 0.3872690258145332
Test accuracy: 0.9037


In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__100_.h5")

sgd = keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)


reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1, 
                                  patience=5, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.001)




model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                    callbacks=[reduce_lr , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/100,
                    epochs=20 , 
                   verbose = 1 , 
#                    workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))





Epoch 1/20
782/782 [==============================] - 240s 306ms/step - loss: 0.1268 - acc: 0.9557 - val_loss: 0.3831 - val_acc: 0.9070
Epoch 2/20
782/782 [==============================] - 238s 304ms/step - loss: 0.1238 - acc: 0.9566 - val_loss: 0.3676 - val_acc: 0.9091
Epoch 3/20
782/782 [==============================] - 237s 304ms/step - loss: 0.1211 - acc: 0.9570 - val_loss: 0.3865 - val_acc: 0.9062
Epoch 4/20
782/782 [==============================] - 237s 303ms/step - loss: 0.1197 - acc: 0.9585 - val_loss: 0.3870 - val_acc: 0.9057
Epoch 5/20
782/782 [==============================] - 237s 303ms/step - loss: 0.1200 - acc: 0.9568 - val_loss: 0.3863 - val_acc: 0.9045

Epoch 00005: acc improved from 0.95610 to 0.95680, saving model to /gdrive/My Drive/CIFAR_10/__100_.h5
Epoch 6/20
782/782 [==============================] - 237s 303ms/step - loss: 0.1202 - acc: 0.9574 - val_loss: 0.3924 - val_acc: 0.9060
Epoch 7/20
782/782 [==============================] - 237s 304ms/step - loss: 0.

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__100_.h5")

sgd = keras.optimizers.SGD(lr=1.0, decay=1e-6, momentum=0.9, nesterov=True)


reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1, 
                                  patience=2, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.01)


#model_checkpoint

model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                    callbacks=[reduce_lr , csv_logger ] ,
#                     steps_per_epoch= len(x_train)/100,
                    epochs=5 , 
                   verbose = 1 , 
#                    workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))





Epoch 1/5
782/782 [==============================] - 240s 307ms/step - loss: 0.1124 - acc: 0.9594 - val_loss: 0.4045 - val_acc: 0.9059
Epoch 2/5
782/782 [==============================] - 237s 303ms/step - loss: 0.1070 - acc: 0.9617 - val_loss: 0.4026 - val_acc: 0.9069
Epoch 3/5
782/782 [==============================] - 237s 303ms/step - loss: 0.1074 - acc: 0.9620 - val_loss: 0.4031 - val_acc: 0.9077
Epoch 4/5
782/782 [==============================] - 237s 303ms/step - loss: 0.1093 - acc: 0.9615 - val_loss: 0.3968 - val_acc: 0.9082
Epoch 5/5
782/782 [==============================] - 237s 303ms/step - loss: 0.1088 - acc: 0.9604 - val_loss: 0.3989 - val_acc: 0.9080


In [0]:
model.save_weights("/gdrive/My Drive/CIFAR_10/__epochs5_valaccuracy9080__4PM.h5")

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__epochs5_valaccuracy9080__4PM.h5")

sgd = keras.optimizers.SGD(lr=1.0, decay=1e-9, momentum=0.9, nesterov=True)


reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1, 
                                  patience=20, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.001)


#model_checkpoint

model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    callbacks=[reduce_lr , csv_logger ] ,
#                     steps_per_epoch= len(x_train)/100,
                    epochs=30 , 
                   verbose = 1 , 
#                    workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))





Epoch 1/5
1563/1563 [==============================] - 285s 183ms/step - loss: 0.1321 - acc: 0.9533 - val_loss: 0.4043 - val_acc: 0.9068
Epoch 2/5
1563/1563 [==============================] - 282s 180ms/step - loss: 0.1284 - acc: 0.9545 - val_loss: 0.3935 - val_acc: 0.9089
Epoch 3/5
1563/1563 [==============================] - 281s 180ms/step - loss: 0.1317 - acc: 0.9530 - val_loss: 0.4387 - val_acc: 0.8971
Epoch 4/5
1563/1563 [==============================] - 283s 181ms/step - loss: 0.1276 - acc: 0.9538 - val_loss: 0.4143 - val_acc: 0.9042
Epoch 5/5
1563/1563 [==============================] - 282s 181ms/step - loss: 0.1255 - acc: 0.9563 - val_loss: 0.4020 - val_acc: 0.9054


In [0]:
model.save_weights("/gdrive/My Drive/CIFAR_10/epoch5_reduceLR_factor02_patience20_minlr0001_decay1e9.h5")

# 91.06 % Accuracy at 34 Epoch

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__epochs5_valaccuracy9080__4PM.h5")

sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)


filepath = "/gdrive/My Drive/CIFAR_10/__best_val_accuracy__.h5"
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='auto', 
                                period=10)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1, 
                                  patience=17, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.001)


#model_checkpoint

model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    callbacks=[reduce_lr , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/100,
                    epochs=50 , 
                   verbose = 1 , 
#                    workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))





Epoch 1/50
1563/1563 [==============================] - 284s 182ms/step - loss: 0.1301 - acc: 0.9535 - val_loss: 0.4220 - val_acc: 0.9030
Epoch 2/50
1563/1563 [==============================] - 281s 180ms/step - loss: 0.1310 - acc: 0.9538 - val_loss: 0.4199 - val_acc: 0.9043
Epoch 3/50
1563/1563 [==============================] - 282s 180ms/step - loss: 0.1282 - acc: 0.9542 - val_loss: 0.4010 - val_acc: 0.9055
Epoch 4/50
1563/1563 [==============================] - 283s 181ms/step - loss: 0.1298 - acc: 0.9538 - val_loss: 0.3870 - val_acc: 0.9092
Epoch 5/50
1563/1563 [==============================] - 284s 181ms/step - loss: 0.1304 - acc: 0.9538 - val_loss: 0.4026 - val_acc: 0.9038
Epoch 6/50
1563/1563 [==============================] - 283s 181ms/step - loss: 0.1273 - acc: 0.9557 - val_loss: 0.4113 - val_acc: 0.9054
Epoch 7/50
1563/1563 [==============================] - 284s 182ms/step - loss: 0.1252 - acc: 0.9553 - val_loss: 0.4229 - val_acc: 0.9023
Epoch 8/50
1563/1563 [============

# 91.23%

In [0]:
from datetime import datetime
time = str(datetime.now())


model.load_weights("/gdrive/My Drive/CIFAR_10/__best_val_accuracy__.h5")
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)


filepath = f"/gdrive/My Drive/CIFAR_10/__oct23_best_val_accuracy__{str(datetime.now())}__.h5"
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='auto', 
                                period=4)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.1, 
                                  patience=17, 
                                  verbose=1, 
                                  mode='auto', 
                                  min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.001)


# earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', 
# #                                           min_delta=6.0, 
#                                           verbose=1, 
#                                           mode='max')








#######################################

class EarlyStoppingByLossVal(keras.callbacks.Callback):
    def __init__(self, monitor='loss', value=0.01, verbose=0):
        super(keras.callbacks.Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            print("Early stopping requires %s available!" % self.monitor)
            exit()

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True
            
########################################

earlystop = EarlyStoppingByLossVal(monitor = "val_acc" , value = 92 , verbose = 1)
csv_logger = keras.callbacks.CSVLogger(filename =  "22_455AM__.csv", separator=',', append=False)


# model.fit(x_train, y_train,
#                     callbacks=[reduce_lr , csv_logger , model_checkpoint ] ,
#                     batch_size=64,
#                     epochs=100,
#                     verbose=1,
#                     validation_data=(x_test, y_test))






# score = model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

# model.save_weights(f"/gdrive/My Drive/CIFAR_10/__23_456AM__{score[1]}.h5")
# print("model saved")





# Trying to reach from 91 to 92. 

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__oct23_best_val_accuracy__2018-10-21 23:28:59.643586__.h5")

sgd = keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)


reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.2, 
                                  patience=1, 
                                  verbose=1, 
                                  mode='auto', 
#                                   min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.0001)

model.fit(x_train, y_train,
                    callbacks=[reduce_lr , csv_logger , model_checkpoint ] ,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))





Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 232s 5ms/step - loss: 0.0050 - acc: 0.9986 - val_loss: 0.5394 - val_acc: 0.9141
Epoch 2/10
50000/50000 [==============================] - 230s 5ms/step - loss: 0.0051 - acc: 0.9987 - val_loss: 0.5357 - val_acc: 0.9140
Epoch 3/10
50000/50000 [==============================] - 231s 5ms/step - loss: 0.0047 - acc: 0.9988 - val_loss: 0.5440 - val_acc: 0.9131
Epoch 4/10
47296/50000 [===========================>..] - ETA: 11s - loss: 0.0051 - acc: 0.9986

KeyboardInterrupt: ignored

# reduced the learning rate. To check whether the training accuracy will cross 99% in the above cell. 

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__oct23_best_val_accuracy__2018-10-21 23:28:59.643586__.h5")

sgd = keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)


reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.2, 
                                  patience=1, 
                                  verbose=1, 
                                  mode='auto', 
#                                   min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.00001)

#reduce_lr
model.fit(x_train, y_train,
                    callbacks=[ csv_logger , model_checkpoint ] ,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))





Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 217s 4ms/step - loss: 0.0052 - acc: 0.9985 - val_loss: 0.5524 - val_acc: 0.9129
Epoch 2/10
50000/50000 [==============================] - 216s 4ms/step - loss: 0.0050 - acc: 0.9987 - val_loss: 0.5720 - val_acc: 0.9106
Epoch 3/10
50000/50000 [==============================] - 216s 4ms/step - loss: 0.0053 - acc: 0.9987 - val_loss: 0.5469 - val_acc: 0.9115
Epoch 4/10
50000/50000 [==============================] - 216s 4ms/step - loss: 0.0051 - acc: 0.9986 - val_loss: 0.5494 - val_acc: 0.9122

Epoch 00004: acc improved from 0.98836 to 0.99860, saving model to /gdrive/My Drive/CIFAR_10/__100_.h5
Epoch 5/10
50000/50000 [==============================] - 216s 4ms/step - loss: 0.0049 - acc: 0.9987 - val_loss: 0.5439 - val_acc: 0.9154
Epoch 6/10
50000/50000 [==============================] - 216s 4ms/step - loss: 0.0054 - acc: 0.9986 - val_loss: 0.5543 - val_acc: 0.9121
Epoch 7/10
50000/5

# want to increase val accuracy

In [0]:
from datetime import datetime

model.load_weights("/gdrive/My Drive/CIFAR_10/__oct23_best_val_accuracy__2018-10-21 23:28:59.643586__.h5")

sgd = keras.optimizers.SGD(lr=3, decay=1e-6, momentum=0.9, nesterov=True)

filepath = f"/gdrive/My Drive/CIFAR_10/__oct23_best_val_accuracy__{str(datetime.now())}__.h5"
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='auto', 
                                period=4)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.2, 
                                  patience=1, 
                                  verbose=1, 
                                  mode='auto', 
#                                   min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.00001)

#reduce_lr
model.fit(x_train, y_train,
                    callbacks=[reduce_lr ,  csv_logger , model_checkpoint ] ,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))





Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 218s 4ms/step - loss: 0.0052 - acc: 0.9985 - val_loss: 0.5491 - val_acc: 0.9133
Epoch 2/10
50000/50000 [==============================] - 218s 4ms/step - loss: 0.0051 - acc: 0.9987 - val_loss: 0.5474 - val_acc: 0.9133
Epoch 3/10
50000/50000 [==============================] - 218s 4ms/step - loss: 0.0048 - acc: 0.9987 - val_loss: 0.5561 - val_acc: 0.9126

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.
Epoch 4/10
50000/50000 [==============================] - 218s 4ms/step - loss: 0.0050 - acc: 0.9986 - val_loss: 0.5440 - val_acc: 0.9145

Epoch 00004: val_acc improved from -inf to 0.91450, saving model to /gdrive/My Drive/CIFAR_10/__oct23_best_val_accuracy__2018-10-22 10:22:32.831683__.h5
Epoch 5/10
50000/50000 [==============================] - 216s 4ms/step - loss: 0.0044 - acc: 0.9989 - val_loss: 0.5454 - val_acc: 0.9137

Epoch 00005: ReduceLROnP

# increase val accuracy with data augmentation - model fit generator


In [0]:
from datetime import datetime

model.load_weights("/gdrive/My Drive/CIFAR_10/__oct23_best_val_accuracy__2018-10-21 23:28:59.643586__.h5")

sgd = keras.optimizers.SGD(lr=1.0, decay=1e-6, momentum=0.9, nesterov=True)

filepath = f"/gdrive/My Drive/CIFAR_10/__oct23_best_val_accuracy__{str(datetime.now())}__.h5"
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='auto', 
                                period=4)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.2, 
                                  patience=1, 
                                  verbose=1, 
                                  mode='auto', 
#                                   min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.00001)



datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    brightness_range = [0.1 , 0.5],
    rotation_range=15,
    width_shift_range=0.125,
    height_shift_range=0.125,
    fill_mode='constant',
    zoom_range=0.2,
    horizontal_flip=True)


datagen.fit(x_train)


model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    callbacks=[reduce_lr , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/100,
                    epochs=10 , 
                   verbose = 1 , 
#                    workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))


NameError: ignored

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__oct23_best_val_accuracy__2018-10-21 23:28:59.643586__.h5")

sgd = keras.optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.2, 
                                  patience=1, 
                                  verbose=1, 
                                  mode='auto', 
#                                   min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.00001)

#reduce_lr
model.fit(x_train, y_train,
                    callbacks=[  csv_logger , model_checkpoint ] ,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))





Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 243s 5ms/step - loss: 0.0053 - acc: 0.9985 - val_loss: 0.5366 - val_acc: 0.9148
Epoch 2/10
50000/50000 [==============================] - 232s 5ms/step - loss: 0.0047 - acc: 0.9987 - val_loss: 0.5360 - val_acc: 0.9149
Epoch 3/10
50000/50000 [==============================] - 233s 5ms/step - loss: 0.0044 - acc: 0.9991 - val_loss: 0.5349 - val_acc: 0.9149
Epoch 4/10
50000/50000 [==============================] - 233s 5ms/step - loss: 0.0043 - acc: 0.9990 - val_loss: 0.5327 - val_acc: 0.9147
Epoch 5/10
50000/50000 [==============================] - 233s 5ms/step - loss: 0.0044 - acc: 0.9988 - val_loss: 0.5358 - val_acc: 0.9151

Epoch 00005: acc improved from -inf to 0.99882, saving model to /gdrive/My Drive/CIFAR_10/__100_.h5
Epoch 6/10
50000/50000 [==============================] - 234s 5ms/step - loss: 0.0047 - acc: 0.9988 - val_loss: 0.5367 - val_acc: 0.9145
Epoch 7/10
50000/5000

# Increased initial Learning rate in SGD.

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__oct23_best_val_accuracy__2018-10-21 23:28:59.643586__.h5")

sgd = keras.optimizers.SGD(lr=1.0, decay=1e-3, momentum=0.3, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.2, 
                                  patience=1, 
                                  verbose=1, 
                                  mode='auto', 
#                                   min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.00001)

#reduce_lr
model.fit(x_train, y_train,
                    callbacks=[  csv_logger , model_checkpoint ] ,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))


#0.0003


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 241s 5ms/step - loss: 1.9566 - acc: 0.3168 - val_loss: 2.1539 - val_acc: 0.3153
Epoch 2/10
50000/50000 [==============================] - 232s 5ms/step - loss: 1.2380 - acc: 0.5493 - val_loss: 1.2703 - val_acc: 0.5646
Epoch 3/10
50000/50000 [==============================] - 232s 5ms/step - loss: 0.9266 - acc: 0.6672 - val_loss: 1.0416 - val_acc: 0.6291
Epoch 4/10
50000/50000 [==============================] - 233s 5ms/step - loss: 0.7654 - acc: 0.7285 - val_loss: 0.8721 - val_acc: 0.7005
Epoch 5/10
50000/50000 [==============================] - 232s 5ms/step - loss: 0.6632 - acc: 0.7660 - val_loss: 1.1580 - val_acc: 0.6476

Epoch 00005: acc improved from -inf to 0.76602, saving model to /gdrive/My Drive/CIFAR_10/__100_.h5
Epoch 6/10
50000/50000 [==============================] - 232s 5ms/step - loss: 0.5805 - acc: 0.7951 - val_loss: 0.7414 - val_acc: 0.7534
Epoch 7/10
50000/5000

# continuing above execution with extra 10 epochs

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__100_.h5")

sgd = keras.optimizers.SGD(lr=1.0, decay=1e-3, momentum=0.3, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.2, 
                                  patience=1, 
                                  verbose=1, 
                                  mode='auto', 
#                                   min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.00001)

#reduce_lr
model.fit(x_train, y_train,
                    callbacks=[  csv_logger , model_checkpoint ] ,
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))


#0.0003


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 282s 6ms/step - loss: 0.7174 - acc: 0.7542 - val_loss: 0.7687 - val_acc: 0.7469
Epoch 2/10
50000/50000 [==============================] - 271s 5ms/step - loss: 0.4818 - acc: 0.8347 - val_loss: 0.5174 - val_acc: 0.8321
Epoch 3/10
50000/50000 [==============================] - 271s 5ms/step - loss: 0.3913 - acc: 0.8661 - val_loss: 0.5272 - val_acc: 0.8344
Epoch 4/10
50000/50000 [==============================] - 274s 5ms/step - loss: 0.3386 - acc: 0.8830 - val_loss: 0.4464 - val_acc: 0.8595
Epoch 5/10
50000/50000 [==============================] - 272s 5ms/step - loss: 0.2976 - acc: 0.8974 - val_loss: 0.4653 - val_acc: 0.8591

Epoch 00005: acc improved from 0.85644 to 0.89740, saving model to /gdrive/My Drive/CIFAR_10/__100_.h5
Epoch 6/10
50000/50000 [==============================] - 272s 5ms/step - loss: 0.2676 - acc: 0.9070 - val_loss: 0.4419 - val_acc: 0.8667
Epoch 7/10
50000/5

# increasing the epochs

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/__100_.h5")

sgd = keras.optimizers.SGD(lr=0.01, decay=1e-3, momentum=0.3, nesterov=False)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.2, 
                                  patience=1, 
                                  verbose=1, 
                                  mode='auto', 
#                                   min_delta=0.0001, 
                                  cooldown=0, 
                                  min_lr=0.00001)

#reduce_lr
model.fit(x_train, y_train,
                    callbacks=[  csv_logger , model_checkpoint ] ,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))


#0.0003


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 240s 5ms/step - loss: 0.1497 - acc: 0.9482 - val_loss: 0.4145 - val_acc: 0.8890
Epoch 2/10
50000/50000 [==============================] - 230s 5ms/step - loss: 0.1426 - acc: 0.9503 - val_loss: 0.4122 - val_acc: 0.8904
Epoch 3/10
50000/50000 [==============================] - 229s 5ms/step - loss: 0.1412 - acc: 0.9510 - val_loss: 0.4145 - val_acc: 0.8890
Epoch 4/10
50000/50000 [==============================] - 229s 5ms/step - loss: 0.1387 - acc: 0.9518 - val_loss: 0.4170 - val_acc: 0.8886
Epoch 5/10
50000/50000 [==============================] - 229s 5ms/step - loss: 0.1370 - acc: 0.9519 - val_loss: 0.4156 - val_acc: 0.8890

Epoch 00005: acc improved from 0.93046 to 0.95190, saving model to /gdrive/My Drive/CIFAR_10/__100_.h5
Epoch 6/10
50000/50000 [==============================] - 229s 5ms/step - loss: 0.1362 - acc: 0.9521 - val_loss: 0.4192 - val_acc: 0.8890
Epoch 7/10
50000/5

# collecting information from github code

In [0]:
lr_reducer      = ReduceLROnPlateau(monitor='val_acc', factor=np.sqrt(0.1),
                                    cooldown=0, patience=5, min_lr=1e-5)
model_checkpoint= ModelCheckpoint(weights_file, monitor="val_acc", save_best_only=True,
                                  save_weights_only=True, verbose=1)




lr_reducer = ReduceLROnPlateau(monitor='val_acc', factor=np.sqrt(0.1),
                                  cooldown=0, patience=3, min_lr=1e-6)
  """
  model_checkpoint = ModelCheckpoint(check_point_file, monitor="val_acc", save_best_only=True,
                                save_weights_only=True, verbose=1)

In [0]:
from google.colab import drive
drive.mount('/gdrive' )

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
! ls "/gdrive/My Drive/CIFAR_10/__oct23_best_val_accuracy__2018-10-21 23:28:59.643586__.h5"

 __100_.h5
 __best_val_accuracy__.h5
 DNST_CIFAR10_AUG.ipynb
 epoch5_reduceLR_factor02_patience20_minlr0001_decay1e9.h5
 __epochs5_valaccuracy9080__4PM.h5
'__oct23_best_val_accuracy__2018-10-21 23:28:59.643586__.h5'


# Oct 22 
---

# 1 - 55 Epochs

In [0]:

#keras library

import keras
import keras.backend as K
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam 
from keras.layers import Input
from keras.callbacks import LearningRateScheduler


# densenet 

def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp
  
  
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg
 




def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output
  
#############################################################################
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 200
l = 12 #L
num_filter = 36 #k
compression = 0.5
dropout_rate = 0.2
#############################################################################
  
  
  
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]




# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



#normalizing data

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255





input = Input(shape=(img_height, img_width, channel))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# Back_Prop_First_Conv2D

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)


Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)



model = Model(inputs=[input], outputs=[output])
# model.summary()



#optimizer
sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  if epoch <= 150:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 150 and epoch <=225:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=5)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=epochs , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))



Epoch 1/200
learning rate ---> 0.1 Epoch ---> 0
782/782 [==============================] - 330s 422ms/step - loss: 2.2196 - acc: 0.2376 - val_loss: 1.8861 - val_acc: 0.3230
Epoch 2/200
learning rate ---> 0.1 Epoch ---> 1
782/782 [==============================] - 303s 388ms/step - loss: 1.7384 - acc: 0.3564 - val_loss: 1.8138 - val_acc: 0.3477
Epoch 3/200
learning rate ---> 0.1 Epoch ---> 2
782/782 [==============================] - 303s 388ms/step - loss: 1.5817 - acc: 0.4188 - val_loss: 1.8889 - val_acc: 0.3938
Epoch 4/200
learning rate ---> 0.1 Epoch ---> 3
782/782 [==============================] - 302s 387ms/step - loss: 1.4471 - acc: 0.4685 - val_loss: 1.3868 - val_acc: 0.5036
Epoch 5/200
learning rate ---> 0.1 Epoch ---> 4
782/782 [==============================] - 302s 387ms/step - loss: 1.3378 - acc: 0.5141 - val_loss: 1.3788 - val_acc: 0.5284

Epoch 00005: val_acc improved from -inf to 0.52840, saving model to /gdrive/My Drive/CIFAR_10/weights.05-0.53.hdf5
Epoch 6/200
learnin

# 56 - 100 Epoch

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/weights.55-0.87.hdf5")

sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  
  epoch += 55 
  if epoch <= 150:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 150 and epoch <=225:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=5)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=45 , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))

Epoch 1/45
learning rate ---> 0.1 Epoch ---> 55
782/782 [==============================] - 310s 396ms/step - loss: 0.3207 - acc: 0.8883 - val_loss: 0.6701 - val_acc: 0.8259
Epoch 2/45
learning rate ---> 0.1 Epoch ---> 56
782/782 [==============================] - 298s 381ms/step - loss: 0.3173 - acc: 0.8905 - val_loss: 0.4855 - val_acc: 0.8667
Epoch 3/45
learning rate ---> 0.1 Epoch ---> 57
782/782 [==============================] - 298s 381ms/step - loss: 0.3095 - acc: 0.8931 - val_loss: 0.7172 - val_acc: 0.8320
Epoch 4/45
learning rate ---> 0.1 Epoch ---> 58
782/782 [==============================] - 298s 381ms/step - loss: 0.3087 - acc: 0.8924 - val_loss: 0.4474 - val_acc: 0.8726
Epoch 5/45
learning rate ---> 0.1 Epoch ---> 59
782/782 [==============================] - 298s 381ms/step - loss: 0.3043 - acc: 0.8946 - val_loss: 0.5453 - val_acc: 0.8566

Epoch 00005: val_acc improved from -inf to 0.85660, saving model to /gdrive/My Drive/CIFAR_10/weights.05-0.86.hdf5
Epoch 6/45
learning

# 100 - 150 Epoch

In [8]:
model.load_weights("/gdrive/My Drive/CIFAR_10/weights.45-0.90.hdf5")

sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  
  epoch += 100 
  if epoch <= 120:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 120 and epoch <=210:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=45 , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))

Epoch 1/45
learning rate ---> 0.1 Epoch ---> 100
782/782 [==============================] - 311s 398ms/step - loss: 0.2192 - acc: 0.9221 - val_loss: 0.3970 - val_acc: 0.8907

Epoch 00001: val_acc improved from -inf to 0.89070, saving model to /gdrive/My Drive/CIFAR_10/weights.01-0.89.hdf5
Epoch 2/45
learning rate ---> 0.1 Epoch ---> 101
782/782 [==============================] - 297s 380ms/step - loss: 0.2178 - acc: 0.9235 - val_loss: 0.4981 - val_acc: 0.8796

Epoch 00002: val_acc did not improve from 0.89070
Epoch 3/45
learning rate ---> 0.1 Epoch ---> 102
782/782 [==============================] - 297s 380ms/step - loss: 0.2166 - acc: 0.9241 - val_loss: 0.3381 - val_acc: 0.8995

Epoch 00003: val_acc improved from 0.89070 to 0.89950, saving model to /gdrive/My Drive/CIFAR_10/weights.03-0.90.hdf5
Epoch 4/45
learning rate ---> 0.1 Epoch ---> 103
782/782 [==============================] - 298s 381ms/step - loss: 0.2136 - acc: 0.9255 - val_loss: 0.3725 - val_acc: 0.9003

Epoch 00004: val_

# 150 - 180 Epochs

In [0]:
model.load_weights("/gdrive/My Drive/CIFAR_10/weights.45-0.92.hdf5")

sgd = keras.optimizers.SGD(lr=0.1 ,  momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#callbacks


def lr_schedule(epoch):
  
  epoch += 150 
  if epoch <= 120:
    K.set_value(model.optimizer.lr, 0.1)
    print("learning rate --->" , 0.1 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  elif epoch > 120 and epoch <=210:
    K.set_value(model.optimizer.lr, 0.01)
    print("learning rate --->" , 0.01 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
  else:
    K.set_value(model.optimizer.lr, 0.001)
    print("learning rate --->" , 0.001 , "Epoch --->" , epoch)
    return K.get_value(model.optimizer.lr)
    
      
  
lr_reducer = LearningRateScheduler(lr_schedule)

filepath = '/gdrive/My Drive/CIFAR_10/weights.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath,
                                monitor='val_acc', 
                                verbose=1, 
                                save_best_only=True, 
                                save_weights_only=False, 
                                mode='max', 
                                period=1)


csv_logger = keras.callbacks.CSVLogger(filename =  "fifth__.csv", separator=',', append=False)


datagen = keras.preprocessing.image.ImageDataGenerator(
#     featurewise_std_normalization=True,
#     rotation_range=9 , 
    width_shift_range=0.125,
    height_shift_range=0.125,
    horizontal_flip=True)



datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    callbacks=[lr_reducer , csv_logger , model_checkpoint] ,
#                     steps_per_epoch= len(x_train)/batch_size,
                    epochs=30 , 
                   verbose = 1 , 
                   workers = 4 , 
#                    use_multiprocessing = True , 
                   shuffle = True , 
                   validation_data=(x_test, y_test))

Epoch 1/30
learning rate ---> 0.01 Epoch ---> 150
782/782 [==============================] - 315s 402ms/step - loss: 0.1195 - acc: 0.9584 - val_loss: 0.3483 - val_acc: 0.9168

Epoch 00001: val_acc improved from -inf to 0.91680, saving model to /gdrive/My Drive/CIFAR_10/weights.01-0.92.hdf5
Epoch 2/30
learning rate ---> 0.01 Epoch ---> 151
782/782 [==============================] - 298s 381ms/step - loss: 0.1170 - acc: 0.9584 - val_loss: 0.3473 - val_acc: 0.9173

Epoch 00002: val_acc improved from 0.91680 to 0.91730, saving model to /gdrive/My Drive/CIFAR_10/weights.02-0.92.hdf5
Epoch 3/30
learning rate ---> 0.01 Epoch ---> 152
782/782 [==============================] - 298s 381ms/step - loss: 0.1156 - acc: 0.9589 - val_loss: 0.3588 - val_acc: 0.9165

Epoch 00003: val_acc did not improve from 0.91730
Epoch 4/30
learning rate ---> 0.01 Epoch ---> 153
782/782 [==============================] - 298s 381ms/step - loss: 0.1175 - acc: 0.9584 - val_loss: 0.3484 - val_acc: 0.9159

Epoch 00004: 